In [60]:
from neo4j import GraphDatabase, basic_auth
import nxneo4j as nx #using nxneo4j
from py2neo import Graph, Node 
import pandas

In [61]:
cGraph = Graph("neo4j+s://101fd6b7.databases.neo4j.io", auth=('neo4j', "gB9F-fD2doYqInIcXR3DJZwnvvDWm-ZpgvOJ3BGCl54"))

In [59]:
recipeList = pandas.read_csv('../CSV/recipeList.csv')
listOfRecipes = pandas.read_csv('../CSV/recipeLinks.csv')

In [62]:
uniqueIng = []

for index, row in recipeList.iterrows():
    temp = [row[1].replace("_", " "), row[2]]
    if temp not in uniqueIng:
        uniqueIng.append(temp)
        

In [64]:
uniqueRecipe = []

for index, row in listOfRecipes.iterrows():
    temp = [row[0], row[1]]
    if temp not in uniqueRecipe:
        uniqueRecipe.append(temp)

In [84]:
for currIng, ingType in uniqueIng:
    currIng = currIng.replace("'", "-")
    addNode = "CREATE (n:Ingredient {name : '" + currIng + "', type: '" + str(ingType) + "'})"
        
    cGraph.query(addNode)

In [85]:
for recipe in uniqueRecipe:
    addNode = "CREATE (n:Recipe {name : '" + recipe[0].replace("'", "-") + "', link : '" + recipe[1] + "'})"
    cGraph.query(addNode)

In [31]:
ingPairWeights = pandas.read_csv('../CSV/ingPairWeights.csv')

In [32]:
counter = 0
tempArray = []
splitIngPairWeights = []
for index, row in ingPairWeights.iterrows():
    if counter == 3000:
        splitIngPairWeights.append(tempArray)
        tempArray = []
        counter = 0

    tempArray.append(row)
    counter += 1

splitIngPairWeights.append(tempArray)

In [ ]:
iterCounter = 0 # change to the next index before you run (currently at 0) (up to 13)

for counter in range(0, len(splitIngPairWeights[iterCounter])): 
    splitIngPairWeights[iterCounter][counter][0] = splitIngPairWeights[iterCounter][counter][0].replace("'", "-")
    splitIngPairWeights[iterCounter][counter][1] = splitIngPairWeights[iterCounter][counter][1].replace("'", "-")

    exists = cGraph.query("MATCH ({name : '" + splitIngPairWeights[iterCounter][counter][0] + "'})-[r]-({name : '" + splitIngPairWeights[iterCounter][counter][1] + "'}) RETURN r")
    exists = exists.data()

    if len(exists) == 0 and (splitIngPairWeights[iterCounter][counter][2] != 0 or splitIngPairWeights[iterCounter][counter][3] != 0):
        addEdge = "MATCH (a:Ingredient), (b:Ingredient) WHERE a.name = '" + splitIngPairWeights[iterCounter][counter][0] + "' AND b.name = '" + splitIngPairWeights[iterCounter][counter][1] + "' CREATE (a)-[r:IngPair {comRecWeight:" + str(splitIngPairWeights[iterCounter][counter][2]) + ", comCompWeight:" + str(splitIngPairWeights[iterCounter][counter][3]) + "}]->(b) RETURN type(r)"
        cGraph.query(addEdge)

In [ ]:
# for index, row in ingPairWeights.iterrows():
#     row[0] = row[0].replace("'", "-")
#     row[1] = row[1].replace("'", "-")

#     exists = cGraph.query("MATCH ({name : '" + row[0] + "'})-[r]-({name : '" + row[1] + "'}) RETURN r")
#     exists = exists.data()

#     if len(exists) == 0 and (row[2] != 0 or row[3] != 0):
#         addEdge = "MATCH (a:Ingredient), (b:Ingredient) WHERE a.name = '" + row[0] + "' AND b.name = '" + row[1] + "' CREATE (a)-[r:IngPair {comRecWeight:" + str(row[2]) + ", comCompWeight:" + str(row[3]) + "}]->(b) RETURN type(r)"
#         cGraph.query(addEdge)

In [83]:
recipeIngredients = []

for counter in range(0, len(recipeList)):
    entry = [recipeList['Recipe'][counter].replace("'", "-"), recipeList['Ingredient'][counter].replace("'", "-")]
    if entry not in recipeIngredients:
        recipeIngredients.append(entry)

print(len(recipeIngredients))

1501


In [86]:
for entry in recipeIngredients:
    addEdge = "MATCH (a:Recipe), (b:Ingredient) WHERE a.name = '" + entry[0] + "' AND b.name = '" + entry[1] + "' CREATE (a)-[r:RecIng]->(b) RETURN type(r)"
    cGraph.query(addEdge)

In [ ]:
for index, row in recipeList.iterrows():
    row[0] = row[0].replace("'", "-")
    row[1] = row[1].replace("'", "-")

    exists = cGraph.query("MATCH ({name : '" + row[0] + "'})-[r]-({name : '" + row[1] + "'}) RETURN r")
    exists = exists.data()

    if len(exists) == 0:
        addEdge = "MATCH (a:Recipe), (b:Ingredient) WHERE a.name = '" + row[0] + "' AND b.name = '" + row[1] + "' CREATE (a)-[r:RecIng]->(b) RETURN type(r)"
        cGraph.query(addEdge)